# To INSTALL packages

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy

!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 1.7 MB 9.5 MB/s 
     |████████████████████████████████| 958 kB 50.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 6.0 MB 7.1 MB/s            
     |████████████████████████████████| 42 kB 1.3 MB/s             
     |████████████████████████████████| 181 kB 59.1 MB/s            
     |████████████████████████████████| 451 kB 58.7 MB/s            
     |████████████████████████████████| 10.1 MB 45.1 MB/s            
     |████████████████████████████████| 628 kB 59.0 MB/s            
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uni

In [2]:
import pandas as pd
import re

# Load trained Entity Recognition model in Portuguese based on the references:
# https://monkeylearn.com/blog/named-entity-recognition/
# https://spacy.io/usage/linguistic-features#tokenization
import spacy
nlp = spacy.load("pt_core_news_sm")

from itertools import chain

import numpy as np

# To read data

In [4]:
# Google drive conection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Names associated to a gender
# https://brasil.io/dataset/genero-nomes/nomes/

path_dados = "/content/drive/MyDrive/INSPER/03_trabfinal_1certif/DATA/"

names = pd.read_csv(path_dados + 'nomes.csv')  
names_aux = names[['first_name', 'classification']]

In [6]:
# News texts

path_proc = "/content/drive/MyDrive/INSPER/03_trabfinal_1certif/DATA/Processed/"

from os import listdir
files = listdir(path_proc)
files

['1721034292748976_Isac Godinho_2022-01-06_Folhajus.txt',
 '1721167833180765_Wesley Faraó Klimpel_2022-01-05_Mobilidade Urbana.txt',
 '1721155128269049_Isabella Menon_2022-01-05_Coronavírus.txt',
 '1721153153464175_José Matheus Santos_2022-01-05_Coronavírus.txt',
 '1721122587219657_Luís Costa_2022-01-05_Rio de Janeiro.txt',
 '1721134973363736_João Pedro Pitombo_2022-01-05_Coronavírus.txt',
 '1721130095679618_Fábio Pescarini_2022-01-05_Coronavírus.txt']

# Functions

In [7]:
# This function returns the number of quotes found at a sentence
def regex_quote(string):
    pattern1 = re.compile(r'"{1}') 
    result1 = re.search(r'"{1}', string)
    result2 = re.search(r'["][\w\s]+["]', string)

    if result1:
      return 1
    elif result2:
      return 2
    return 0


# To return the index of the sentence list that have at least one quote
def condition_x_gt_0(x): return x > 0


# To return a vector that has 1 at the entry that has an entity recognized and 0 otherwise
def empty_part(part):
  if part:
      return 1
  return 0


# Difference (in absolute value) between each pair of sentence indexes
def dif_pair(pair): return abs(pair[0] - pair[1])






def read_file_save_entities_count(PATH_NEWSTEXT, CHOSEN_FILE, NLP, NAMES_GENDER, PATH_SAVE_ENTITIES):
  # File name has parameters associated with the text
  params_chosen_file = CHOSEN_FILE.split("_")
  numchar_edit = len(params_chosen_file[3])
  par = {'articleId': params_chosen_file[0], 
     'author': params_chosen_file[1],
     'date': params_chosen_file[2],
     'edit': params_chosen_file[3][0:(numchar_edit-4)]
     }
     
  df_param = pd.DataFrame(data = par, index = [params_chosen_file[0]])

  # To read the data
  file1 = open(PATH_NEWSTEXT + CHOSEN_FILE, "r+")
  text = file1.read()
  
  # Sentence tokenization
  paragraphs = text.split('    ')

  str_in_doublequotes = r'"[^"]*\."|("[^"]*")*([^".]*\.)'
  sentences = []
  for pg in paragraphs:
    in_quotes = re.findall(str_in_doublequotes, pg)
    for grp in in_quotes:
      aux = "".join(grp)
      aux_trim = aux.strip()
      sentences.append(aux_trim)
  
  sent_numb_quotes = [regex_quote(sent) for sent in sentences]


  # Number of quotes found by sentence 
  index_quote = [idx for idx, element in enumerate(sent_numb_quotes) if condition_x_gt_0(element)]  

  # Apply entity recognition model by sentence & Keep only the PERSON Entity entries 
  entities = []
  for sent in sentences:
    sent_ent = NLP(sent)
    # Entity recognition
    ents = [(e.text, e.label_) for e in sent_ent.ents if e.label_ == "PER"]
    entities.append(ents)  

  # Unnest Entities on another list
  entities_list = list(chain.from_iterable(entities))

  # Entities on a data frame
  df_entities = pd.DataFrame(entities_list, columns = ['entity', 'type'])

  # Binary vector for entries with entity recognized 
  sent_entity = [empty_part(part) for part in entities]

  # Which sentences have " (double quote) ?
  # To return the index of the sentence list that has at least one entity recognized
  index_ent = [idx for idx, element in enumerate(sent_entity) if condition_x_gt_0(element)]

  # Update 'df_entities' with the corresponding sentence index
  df_index_ent = pd.DataFrame(index_ent, columns = ['entity_index'])

  df_entities = pd.concat([df_entities, df_index_ent], axis = 1)

  # Filtering Entities that have at least one capital letter
  df_entities = df_entities[df_entities['entity'].str.match("[A-Z]+") == True]

  # Split the name into First and Family Name
  df_entities[['FirstName', 'FamilyName']] = df_entities['entity'].str.split(' ', 1, expand = True)

  # First name in Capital letter
  df_entities['FirstName'] = df_entities['FirstName'].str.upper()

  # Remove accents from the First Name
  df_entities['FirstName'] = df_entities['FirstName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

  # Update df_entities with gender
  df_entities = df_entities.merge(NAMES_GENDER, left_on = 'FirstName', right_on = 'first_name')
  
  # All possible pair combination of 'sentence with quotation index' and 'sentence with entity index'
  pair_idquote_ident = [(x, y) for x in index_quote for y in index_ent]

  # Difference (in absolute value) between each pair of sentence indexes
  dif_pair_idquote_ident = [dif_pair(pair) for pair in pair_idquote_ident]  

  # Convert 'pair_idquote_ident' into numpy array
  pair_idquote_ident_array = np.asarray(pair_idquote_ident)

  # Convert 'dif_pair_idquote_ident' into numpy array
  dif_pair_idquote_ident_array = np.array(dif_pair_idquote_ident)

  # 'cbind' pair_idquote_ident_array and dif_pair_idquote_ident_array
  dif_pair_idquote_ident_dif_array = np.column_stack((pair_idquote_ident_array, 
                                                    dif_pair_idquote_ident_array))
  
  # Convert dif_pair_idquote_ident_dif_array into a long format data.frame
  dif_pair_idquote_ident_df = pd.DataFrame(dif_pair_idquote_ident_dif_array, columns = ['quote', 'entity', 'dif'])

  # Pivot long format object to wide format
  dif_pair_idquote_ident_df_pv = dif_pair_idquote_ident_df.pivot(index = 'quote', columns = 'entity', values = 'dif')

  # Column index associated with the minimum value by row 
  pos = dif_pair_idquote_ident_df_pv.apply(np.argmin, axis = 'columns')

  indices_to_access = list(dif_pair_idquote_ident_df_pv.columns[pos])
  d = {'quote': dif_pair_idquote_ident_df_pv.index, 
     'entity_index': indices_to_access}
  
  df_quote = pd.DataFrame(data = d)

  df_entities = df_entities.merge(df_quote, 
                                on = 'entity_index', 
                                how = 'left')
  
  # To save final file
  for row in range(len(df_entities)):
    if row == 0:
      df_param_aux = df_param
    else:    
      df_param_aux = pd.concat([df_param_aux, df_param])


  df_entities_final = pd.concat([df_param_aux.reset_index(drop=True), df_entities.reset_index(drop=True)], axis = 1)

  name_file_tosave = PATH_SAVE_ENTITIES + 'entrecog_' + CHOSEN_FILE  
  df_entities_final.to_csv(name_file_tosave)

# Apply the function to data

In [10]:
for FILE in files:
  read_file_save_entities_count(PATH_NEWSTEXT = "/content/drive/MyDrive/INSPER/03_trabfinal_1certif/DATA/Processed/", 
                              CHOSEN_FILE = FILE, 
                              NLP = nlp, 
                              NAMES_GENDER = names_aux, 
                              PATH_SAVE_ENTITIES = "/content/drive/MyDrive/INSPER/03_trabfinal_1certif/DATA/Entities/")

In [9]:
#import nltk
#nltk.download('punkt')

In [10]:
#from nltk.tokenize import sent_tokenize
## Split sentences by '.'  
#sentences2 = sent_tokenize(text)
#sentences2